# Solving RCPSP with local search/metaheuristics/genetic algorithm

One big family of combinatorial optimisation solver is local search. We include all metaheuristics and genetic algorithm into this simple terminology. 

In general, a local search algorithm explore the solution space by applying local changes to the current set of solutions.

In the case of RCPSP, we have seen in [the first notebook](RCPSP%20%231%20Introduction.ipynb) that we can represent a solution with a priority list of tasks (equivalent to a permutation). 
Therefore local search algorithms are available on the search space being the ensemble of all permutation of tasks. We can imagine many kind of local changes possible to explore the permutation set.

Local search (LS) algorithms are anytime algorithm, we have access to the current best solution whenever we want to stop the optimisation process. LS can't prove that a solution is optimal but it is rarely an issue in real world applications.

## Prerequisites

Concerning the python kernel to use for this notebook:
- If running locally, be sure to use an environment with discrete-optimization and minizinc.
- If running on colab, the next cell does it for you.
- If running on binder, the environment should be ready.


In [ ]:
# On Colab: install the library
on_colab = "google.colab" in str(get_ipython())
if on_colab:
    import importlib
    import os
    import sys  # noqa: avoid having this import removed by pycln

    !{sys.executable} -m pip install -U pip

    # uninstall google protobuf conflicting with ray and sb3
    ! pip uninstall -y protobuf

    # install dev version for dev doc, or release version for release doc
    !{sys.executable} -m pip install git+https://github.com/airbus/discrete-optimization@master#egg=discrete-optimization

    # be sure to load the proper cffi (downgraded compared to the one initially on colab)
    import cffi

    importlib.reload(cffi)

    # install and configure minizinc
    !curl -o minizinc.AppImage -L https://github.com/MiniZinc/MiniZincIDE/releases/download/2.6.3/MiniZincIDE-2.6.3-x86_64.AppImage
    !chmod +x minizinc.AppImage
    !./minizinc.AppImage --appimage-extract
    os.environ["PATH"] = f"{os.getcwd()}/squashfs-root/usr/bin/:{os.environ['PATH']}"
    os.environ["LD_LIBRARY_PATH"] = (
        f"{os.getcwd()}/squashfs-root/usr/lib/:{os.environ['LD_LIBRARY_PATH']}"
    )

### Imports

In [ ]:
import logging

import matplotlib.pyplot as plt
import nest_asyncio
import numpy as np

from discrete_optimization.datasets import fetch_data_from_psplib
from discrete_optimization.generic_rcpsp_tools.ls_solver import (
    LS_SOLVER,
    LS_RCPSP_Solver,
)
from discrete_optimization.generic_tools.ea.ga import (
    DeapCrossover,
    DeapMutation,
    Ga,
    ObjectiveHandling,
)
from discrete_optimization.generic_tools.ls.hill_climber import (
    HillClimber,
    ModeMutation,
    RestartHandler,
)
from discrete_optimization.generic_tools.ls.local_search import RestartHandlerLimit
from discrete_optimization.generic_tools.ls.simulated_annealing import (
    SimulatedAnnealing,
    TemperatureSchedulingFactor,
)
from discrete_optimization.generic_tools.mutations.mixed_mutation import (
    BasicPortfolioMutation,
)
from discrete_optimization.generic_tools.mutations.mutation_catalog import (
    PermutationMutationRCPSP,
    get_available_mutations,
)
from discrete_optimization.rcpsp.rcpsp_parser import get_data_available, parse_file
from discrete_optimization.rcpsp.solver.rcpsp_pile import GreedyChoice, PileSolverRCPSP

# patch asyncio so that applications using async functions can run in jupyter
nest_asyncio.apply()

# set logging level
logging.basicConfig(level=logging.INFO)

### Download datasets

If not yet available, we import the datasets from [psplib](https://www.om-db.wi.tum.de/psplib/data.html).

In [ ]:
needed_datasets = ["j301_1.sm"]
download_needed = False
try:
    files_available_paths = get_data_available()
    for dataset in needed_datasets:
        if len([f for f in files_available_paths if dataset in f]) == 0:
            download_needed = True
            break
except:
    download_needed = True

if download_needed:
    fetch_data_from_psplib()

In [ ]:
# Parse some rcpsp file
filepath = [f for f in get_data_available() if "j1201_1.sm" in f][0]
rcpsp_problem = parse_file(filepath)

## Hill climbing
The hill climbing is the most basic local search algorithm. It explores the current best solution with a local move we can call `mutation`. If the new generated solution has a better objective value we overwrite the current best solution and repeat the process. The pseudocode is the following.
 
1) Starts from a first solution $s$ and $obj=evaluation(s)$ the objective value of solution $s$ (to minimize)

2) Compute $s'=mutate(s)$ and $obj'=evaluation(s')$

3) if $obj'<obj$ then $(s,obj) \leftarrow (s',obj')$

4) Go back to 2) until some number of iterations.

In [ ]:
HillClimber?

The API of the HillClimber object is the following.
```
HillClimber(
    problem: discrete_optimization.generic_tools.do_problem.Problem,
    mutator: discrete_optimization.generic_tools.do_mutation.Mutation,
    restart_handler: discrete_optimization.generic_tools.ls.local_search.RestartHandler,
    mode_mutation: discrete_optimization.generic_tools.ls.local_search.ModeMutation,
    params_objective_function: Union[discrete_optimization.generic_tools.do_problem.ParamsObjectiveFunction, NoneType] = None,
    store_solution=False,
    nb_solutions=1000)
```
We need to define a ```Mutation``` used in the algorithms.
```RestartHandler``` is not used in HillClimbing, but will be described later in this notebook.

In [ ]:
# Let's compute the available mutation for our RCPSP Problem.
_, mutations = get_available_mutations(rcpsp_problem)
print(len(mutations), " available for the problem ")

We don't want to choose the mutation that we'll me using, we will build a "meta" mutation object that will in practice pick one of the mutation at each iteration (randomly). We call this meta mutation `BasicPortFolio` and can be initialized like this.

In [ ]:
dummy = rcpsp_problem.get_dummy_solution()
list_mutation = [
    mutate[0].build(rcpsp_problem, dummy, **mutate[1])
    for mutate in mutations
    if mutate[0] == PermutationMutationRCPSP
]
mixed_mutation = BasicPortfolioMutation(
    list_mutation, weight_mutation=np.ones((len(list_mutation)))
)

We can now initialize the HillClimbing solver.

In [ ]:
solver = HillClimber(
    problem=rcpsp_problem,
    mode_mutation=ModeMutation.MUTATE_AND_EVALUATE,
    mutator=mixed_mutation,
    restart_handler=RestartHandler(),
)

Let's run the solver : 

In [ ]:
solver.solve??

In [ ]:
results_hc = solver.solve(
    initial_variable=dummy, nb_iteration_max=10000, max_time_seconds=20
)

Let's plot the evolution of the solution found by the algorithm : 

In [ ]:
fig, ax = plt.subplots(1)
ax.plot([x[1] for x in results_hc.list_solution_fits], marker="o")
ax.set_ylabel("- makespan")
ax.set_xlabel("# solution found")
plt.show()

We improved a little bit the dummy solution :-)

## Simulated annealing
[Simulated annealing](https://en.wikipedia.org/wiki/Simulated_annealing) is an algorithm from the class of metaheuristics. It is a famous method to escape local optima of a function because contrary to the Hill Climbing algorithm, the current solution can be different from the current best solution. 
The algorithm is very similar to the HC one, except for the 3) step.

3) if $obj'<obj$ or $e^{\frac{obj-obj'}{T}}\geq random(0,1)$ then $(s, obj)\leftarrow (s',obj')$ 

when $obj'$ is greater than $obj$, the exponent of the exponential is negative. 
When $T$, the temperature setting is high, it is very likely that the $e^{\frac{obj-obj'}{T}}\geq random(0,1)$ inequality is satisfied and that we update our current solution $s$ even though $obj'>obj$.
To the contrary, with a low value $T$, we let less chance for $s'$ to be taken as new current solution. 

- Simulated annealing relies heavily on initial temperature setting and the cooling schedule, which makes the temperature $T$ evolve through the algorithms : in the beginning we can allow high temperature to allow more exploration of the solution space, and then decrease the temperature to focus more on optimisation. in the library it will be done with `TemperatureScheduling` object, that can be custom.
- For simulated annealing, the concept of `RestartHandler` makes sense : it consist in forcing the algorithm to go back to current best solution according to some criteria. Typically if there is no improvement for a given number of iteration, it can make sense to go back to current best solution

In [ ]:
# Come back to best solution every 300 iteration without improvement.
restart_handler = RestartHandlerLimit(
    nb_iteration_no_improvement=300,
)
# Multiply current temperature by a coefficient at every iteration. There might be better cooling schedule
# in the litterature.
temperature_scheduling = TemperatureSchedulingFactor(
    temperature=10, restart_handler=restart_handler, coefficient=0.9999
)

We can now define the simulated annealing solver : 

In [ ]:
solver = SimulatedAnnealing(
    problem=rcpsp_problem,
    mutator=mixed_mutation,
    temperature_handler=temperature_scheduling,
    restart_handler=restart_handler,
    mode_mutation=ModeMutation.MUTATE_AND_EVALUATE,
)

In [ ]:
results_sa = solver.solve(
    initial_variable=dummy, nb_iteration_max=10000, max_time_seconds=20
)

In [ ]:
fig, ax = plt.subplots(1)
ax.plot([x[1] for x in results_sa.list_solution_fits], marker="o")
ax.set_ylabel("- makespan")
ax.set_xlabel("# solution found")
plt.show()

Depending on your luck, you might have found a better solution than using the hillclimbing ;).

Since RCPSP is a widely developed use case in `discrete-optimisation` we developed a wrapper around Hill climbing and simulated annealing to ease their use : 

In [ ]:
solver = LS_RCPSP_Solver(ls_solver=LS_SOLVER.HC, problem=rcpsp_problem)
res = solver.solve(nb_iteration_max=10000, max_time_seconds=20)

In [ ]:
print(res.get_best_solution_fit())

### Starting from a different initial solution
We can use the results of a greedy solver as the initial solution for local search algorithm : 

In [ ]:
solver = PileSolverRCPSP(problem=rcpsp_problem)
res_greedy = solver.solve(greedy_choice=GreedyChoice.MOST_SUCCESSORS)
print("res greedy : ", res_greedy.get_best_solution_fit())
solver = LS_RCPSP_Solver(ls_solver=LS_SOLVER.SA, problem=rcpsp_problem)
res = solver.solve(
    nb_iteration_max=30000,
    max_time_seconds=20,
    starting_point=res_greedy.get_best_solution(),
)
print("local search : ", res.get_best_solution_fit())

## Genetic algorithm

`discrete-optimization` provides wrapper solver to ```deap``` library solver implement evolutionnary algorithms. Genetic algorithms can be seen as a generalisation of previous algorithms, the difference being that the algorithms are now handling a population of solutions instead of only the current one. 
To generate the new population for the current one the genetic algorithm is decomposed in several step : 
- selection : which individuals should be selected for the next generation
- crossover : how to build new individuals from their parent (i.e the individual selected in the previous step)
- mutate : how the new individual from previous step can acquire some mutation (i.e local changes)

The genetic algorithm process is highly inspired by genetic selection and has been proven efficient for a wide range of optimisation problems.

Currently the API of `Ga` is a bit different than others solvers, we need to specify the attribute of our object solution that we consider as a vector representation of the solution : in our case "rcpsp_permutation", we also have to specify manually what is the objective function we want to maximize.

In [ ]:
mutation = DeapMutation.MUT_SHUFFLE_INDEXES
crossover = DeapCrossover.CX_UNIFORM_PARTIALY_MATCHED
ga_solver = Ga(
    rcpsp_problem,
    encoding="rcpsp_permutation",
    objective_handling=ObjectiveHandling.AGGREGATE,
    objectives=["makespan"],
    objective_weights=[-1],
    pop_size=50,
    max_evals=30000,
    mut_rate=0.1,
    crossover_rate=0.9,
    crossover=crossover,
    mutation=mutation,
)
results_ga = ga_solver.solve()

## Conclusion

In this notebook we have shown 3 ways of optimizing the RCPSP problem using local search and evolutionnary algorithms. Most of those solvers are available for all implemented problem in `discrete-optimisation` as long as there is a vector representation of the solution that all those algorithms are able to optimize. The mentionned methods should not be discarded when you have a combinatorial optimisation problem, they're anytime solver and with the right parameters and mutations can become the most suited solution for your problem, especially when the problem is intractable with mathematical methods that we will discuss in next notebook.